Docs:
- https://huggingface.co/docs/optimum-onnx/onnxruntime/package_reference/modeling_ort#optimum.onnxruntime.ORTModelForFeatureExtraction
- https://huggingface.co/optimum/all-MiniLM-L6-v2/tree/main

#### Test with transformers pipeline

In [ ]:
import certifi

certifi.where()

In [ ]:
!uv pip list | grep transformers

In [ ]:
!uv pip list | grep numpy

In [ ]:
!uv pip list | grep optimum

In [ ]:
model_id = "optimum/all-MiniLM-L6-v2"

In [ ]:
from transformers import AutoTokenizer, pipeline



tokenizer = AutoTokenizer.from_pretrained(model_id)

In [ ]:
# from transformers import FeatureExtractionMixin

# extractor = FeatureExtractionMixin.from_pretrained(model_id)

#### Test with onnx runtime

In [ ]:
# from transformers import AutoTokenizer, pipeline
# from optimum.onnxruntime import ORTModelForFeatureExtraction



# tokenizer = AutoTokenizer.from_pretrained(model_id)
# model = ORTModelForFeatureExtraction.from_pretrained(model_id)
# onnx_extractor = pipeline("feature-extraction", model=model, tokenizer=tokenizer)

# text = "My name is Philipp and I live in Germany."
# pred = onnx_extractor(text)